[View in Colaboratory](https://colab.research.google.com/github/wonseokjung/hands_on_tf/blob/master/3_Tensorflow_eager_CNN.ipynb)

In [0]:
import numpy as np

In [0]:
import tensorflow as tf

tf.enable_eager_execution() 



In [0]:
class CNN(tf.keras.Model):
  def __init__(self):
    super().__init__()
    
    self.conv1= tf.keras.layers.Conv2D(
      filters=32, 
      kernel_size = [5,5],
      padding="same",
      activation = tf.nn.relu 
    )
    
    self.pool1 = tf.keras.layers.MaxPool2D(pool_size =[2,2], strides = 2)
    self.conv2=tf.keras.layers.Conv2D(filters = 64,
                                      kernel_size = [5,5],
                                      padding ="same",
                                      activation = tf.nn.relu
                                     )
    self.pool2 = tf.keras.layers.MaxPool2D(pool_size = [2,2], strides = 2)
    
    self.flatten = tf.keras.layers.Reshape(target_shape = (7*7*64,))
    
    self.dense1 = tf.keras.layers.Dense(units = 1024, activation = tf.nn.relu)
   
    
    self.dense2 = tf.keras.layers.Dense(units = 10)
    

    
    
  def call(self, inputs): 
    inputs = tf.reshape(inputs, [-1,28,28,1])
    x = self.conv1(inputs)
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.pool2(x)
    x= self.flatten(x)
    x=self.dense1(x)
    x=self.dense2(x)
    return x
   
    
  def predict(self, inputs):
    logits = self(inputs)
    return tf.argmax(logits, axis = 1 )
  
 
    
    

In [0]:
class DataLoader():
  def __init__(self):
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    
    self.train_data = mnist.train.images 
    self.train_labels = np.asarray(mnist.train.labels, dtype= np.int32)
    self.eval_data = mnist.test.images 
    self.eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
   

  def get_batch(self, batch_size):
    index = np.random.randint(0,np.shape(self.train_data)[0], batch_size)
    
    return self.train_data[index, :], self.train_labels[index]
  
  


In [6]:
model = CNN()
data_loader = DataLoader()

num_batches = 1000
batch_size = 50
learning_rate = 0.001

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)


Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-u

In [7]:
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_logit_pred = model(tf.convert_to_tensor(X))
        loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_logit_pred)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.289092
batch 1: loss 2.227195
batch 2: loss 2.087518
batch 3: loss 1.881793
batch 4: loss 1.714403
batch 5: loss 1.611320
batch 6: loss 1.365711
batch 7: loss 0.936148
batch 8: loss 0.958978
batch 9: loss 0.966190
batch 10: loss 0.739592
batch 11: loss 0.737145
batch 12: loss 0.731559
batch 13: loss 0.555802
batch 14: loss 0.714620
batch 15: loss 0.539785
batch 16: loss 0.366553
batch 17: loss 0.734777
batch 18: loss 0.591951
batch 19: loss 0.348320
batch 20: loss 0.462288
batch 21: loss 0.424636
batch 22: loss 0.467608
batch 23: loss 0.439074
batch 24: loss 0.361790
batch 25: loss 0.744539
batch 26: loss 0.331172
batch 27: loss 0.230847
batch 28: loss 0.348492
batch 29: loss 0.249755
batch 30: loss 0.545975
batch 31: loss 0.420389
batch 32: loss 0.522436
batch 33: loss 0.511936
batch 34: loss 0.398413
batch 35: loss 0.352756
batch 36: loss 0.272183
batch 37: loss 0.265751
batch 38: loss 0.272731
batch 39: loss 0.365220
batch 40: loss 0.439644
batch 41: loss 0.118752
ba

In [8]:
num_eval_samples = np.shape(data_loader.eval_labels)[0]
y_pred = model.predict(data_loader.eval_data).numpy()
print("test accuracy: %f" % (sum(y_pred == data_loader.eval_labels) / num_eval_samples))

test accuracy: 0.986200
